In [7]:
# Don't forget to check website/robots.txt before you start scraping!

# Start some basic scraping
import requests
from bs4 import BeautifulSoup

url = 'http://books.toscrape.com/'
req = requests.get(url)
req

<Response [200]>

In [10]:
# pull in beatiful soup object and get the html. Parse it
soup = BeautifulSoup(req.text, 'html.parser')
soup

<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    All products | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="static/oscar/css/styles.css" rel="stylesheet" type="text/css"/>
<link href="s

In [12]:
# you can pull out html elements by tag name
soup.title.text

'\n    All products | Books to Scrape - Sandbox\n'

In [18]:
soup.title.children
for child in soup.title.children:
    print(child)


    All products | Books to Scrape - Sandbox



In [30]:
#find ul holding category list
li_list = soup.select('ul')

#find list
li_list = li_list[1]

#find all li tags
li_list = li_list.select('li')

#get rid of the outside tags and just get the text inside the a tags
a_list = []
for li in li_list:
    a_list.append(li.a.text.strip())


#this does the same thing as the prvious line but you can do it like this also
categories_list = [li.a.text.strip() for li in li_list]

#print
# a_list
categories_list

['Philosophy',
 'Childrens',
 'Psychology',
 'Autobiography',
 'Christian Fiction',
 'Biography',
 'Thriller',
 'Christian',
 'Short Stories',
 'Health']

In [41]:
#get the title of each book on the page
a_s = soup.findAll('a', {'title':True})

#get the value within the title attribute on each a tag
# for title in titles:
#     print(title['title'])

#or you can do it like this...

#store just the title attribute in a list
titles = [a['title'] for a in a_s]
titles

['A Light in the Attic',
 'Tipping the Velvet',
 'Soumission',
 'Sharp Objects',
 'Sapiens: A Brief History of Humankind',
 'The Requiem Red',
 'The Dirty Little Secrets of Getting Your Dream Job',
 'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull',
 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics',
 'The Black Maria',
 'Starving Hearts (Triangular Trade Trilogy, #1)',
 "Shakespeare's Sonnets",
 'Set Me Free',
 "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)",
 'Rip it Up and Start Again',
 'Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991',
 'Olio',
 'Mesaerion: The Best Science Fiction Stories 1800-1849',
 'Libertarianism for Beginners',
 "It's Only the Himalayas"]

In [55]:
#Make a df out of the information you scrape

import pandas as pd

df = pd.DataFrame(columns=['Title'])

pages = [1,2,3]

for page in pages:
    url = f'http://books.toscrape.com/catalogue/page-{page}.html'
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    
    #get all the article tags with the specific class we mention
    articles = soup.findAll('article', {'class':'product_pod'})
    for article in articles:
        # title = article.h3.a.text.strip()
        # df = df.append({'Title':title}, ignore_index=True)

        #get each url that will take you to a details page and put that in a soup object
        url_details = f'http://books.toscrape.com/catalogue/{article.h3.a["href"]}'
        req_details = requests.get(url_details)
        soup_details = BeautifulSoup(req_details.text, 'html.parser')
        
        upc = soup_details.td.text
        df.loc[upc] = article.h3.a['title']
df

#next try to add in price, stars, quantity in stock, etc.5

,Title
a897fe39b1053632,A Light in the Attic
90fa61229261140a,Tipping the Velvet
6957f44c3847a760,Soumission
e00eb4fd7b871a48,Sharp Objects
4165285e1663650f,Sapiens: A Brief History of Humankind
f77dbf2323deb740,The Requiem Red
2597b5a345f45e1b,The Dirty Little Secrets of Getting Your Dream...
e72a5dfc7e9267b2,The Coming Woman: A Novel Based on the Life of...
e10e1e165dc8be4a,The Boys in the Boat: Nine Americans and Their...
1dfe412b8ac00530,The Black Maria
